In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

In [2]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# Inclusive

In [3]:
expVar: list[str] = ["process", "iteration", "size"]
resVar: str = "Inclusive"

trainDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

testDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

testDF = testDF.reset_index()
trainDF = trainDF.reset_index()

testDF["functionName"] = testDF["Name"]
trainDF["functionName"] = trainDF["Name"]

functionNames: list[str] = sorted(list(set(trainDF["Name"])))

dict_symbols = {}
for elem in expVar:
    dict_symbols[elem] = symbols(elem, real=True)

target_env = [
    (dict_symbols["size"], test_lulesh_sizes[-1]),
    (dict_symbols["iteration"], test_lulesh_iterations[-1]),
    (dict_symbols["process"], test_lulesh_processes[-1]),
]

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

In [4]:
dict_models: dict[str, dict[str, any]] = {}
for functionName in functionNames:

    trainDF_perFunc: pd.DataFrame = trainDF[trainDF["Name"] == functionName]
    testDF_perFunc: pd.DataFrame = testDF[testDF["Name"] == functionName]

    # 総実行時間

    resVar = "Inclusive"
    # Extra-P への入力ファイル作成
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=testDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)
    # Extra-P の実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout
    # 取得した Extra-P の出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)
    # 総実行時間の予測
    model_sympy_all = sympify(res_str, locals=dict_symbols)

    # 1コール当たりの実行時間

    resVar = "InclusivePerCall"
    # Extra-P への入力ファイル作成
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=testDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)
    # Extra-P の実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout
    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)
    # 総実行時間の予測
    model_sympy_perCall = sympify(res_str, locals=dict_symbols)

    # 関数コール回数の予測

    # 関数コール回数予測のためのモデルを構築
    resVar = "#Call"
    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )
    bestModel = bestModelDict["object"]

    dict_models_perFunc: dict[str, any] = {
        "time_all": model_sympy_all,
        "time_perFunc": model_sympy_perCall,
        "call_perFunc": bestModel,
    }
    dict_models[functionName] = dict_models_perFunc

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/sit

In [5]:
print(dict_models_perFunc)
print(f'time_all = {type(dict_models_perFunc["time_all"])}')
print(f'time_perFunc = {type(dict_models_perFunc["time_perFunc"])}')
print(f'call_perFunc = {type(dict_models_perFunc["call_perFunc"])}')

{'time_all': -3.3193430488293343e-13*size**(5/4)*log(iteration)*log(process)/log(2)**2 + 1.7811331690654008e-10*size**(5/4) - 1.1655929556898692e-7*log(process)/log(2) + 1.226624900744705e-6, 'time_perFunc': 3.260324255014677e-16*iteration**3*log(iteration)/log(2) + 2.328094155724892e-12*size**3*log(size)/log(2) + 9.516362036684537e-5, 'call_perFunc': <lib.lab_lib.ModelMultipleEquationForMultipleRegression object at 0x7f11ec384df0>}
time_all = <class 'sympy.core.add.Add'>
time_perFunc = <class 'sympy.core.add.Add'>
call_perFunc = <class 'lib.lab_lib.ModelMultipleEquationForMultipleRegression'>


In [6]:
# 集計をする
# それぞれの予測対象ケースで総実行時間の相対誤差率を算出する
# 算出された相対誤差率をプロットする

resVar: str = "Inclusive"
for elem_process in test_lulesh_processes:
    for elem_iteration in test_lulesh_iterations:
        for elem_size in test_lulesh_sizes:
            testDF_oneCase: pd.DataFrame = ret_averaged_rawDF_lulesh(
                list_process=[elem_process],
                list_iteration=[elem_iteration],
                list_size=[elem_size],
                list_csvDir=list_csvDir,
                resVar=resVar,
            )
            testDF_oneCase["functionName"] = testDF_oneCase["Name"]

            target_env = [
                (dict_symbols["size"], elem_size),
                (dict_symbols["iteration"], elem_iteration),
                (dict_symbols["process"], elem_process),
            ]

            functionNames: list[str] = sorted(list(testDF_oneCase["Name"]))
            list_series: list[pd.Series] = []
            for functionName in functionNames:
                testDF_oneCase_perFunc: pd.DataFrame = testDF_oneCase[
                    testDF_oneCase["Name"] == functionName
                ]

                if len(testDF_oneCase_perFunc) != 1:
                    warnings.warn("len(testDF_oneCase_perFunc) != 1")
                    exit

                # 総実行時間の予測
                Inclusive_predicted_all: float = (
                    dict_models[functionName]["time_all"].subs(target_env).evalf()
                )
                # 1コール当たりの実行時間
                Inclusive_predicted_perFunc: float = (
                    dict_models[functionName]["time_perFunc"].subs(target_env).evalf()
                )
                # コール回数の予測
                predicted_call: float = float(
                    np.array(
                        dict_models[functionName]["call_perFunc"].predict(
                            inputDF=testDF_oneCase_perFunc[expVar]
                        )
                    )
                )

                # 実測値の取得
                Inclusive_real_time: float = testDF_oneCase_perFunc.reset_index().loc[
                    0
                ][resVar]
                real_call: float = testDF_oneCase_perFunc.reset_index().loc[0]["#Call"]

                _series: pd.Series = pd.Series(
                    {
                        "functionName": functionName,
                        f"{resVar}_real_time": Inclusive_real_time,
                        f"{resVar}_predicted_all": Inclusive_predicted_all,
                        f"{resVar}_predicted_from_perCall": Inclusive_predicted_perFunc
                        * predicted_call,
                    }
                )
                list_series.append(_series)

            resultDF_perFunc: pd.DataFrame = pd.concat(list_series, axis=1).T
            resultDF_perFunc = add_relativeErrorRateCol(
                inputDF=resultDF_perFunc,
                real_colName=f"{resVar}_real_time",
                predicted_colName=f"{resVar}_predicted_all",
                targetColName=f"相対誤差率（Extra-P）",
            )
            resultDF_perFunc = add_relativeErrorRateCol(
                inputDF=resultDF_perFunc,
                real_colName=f"{resVar}_real_time",
                predicted_colName=f"{resVar}_predicted_from_perCall",
                targetColName=f"相対誤差率（組合せ）",
            )
            print(resultDF_perFunc)

                                         functionName Inclusive_real_time  \
0                                    .TAU_application          132.158333   
1                                     MPI_Allreduce()           18.807333   
2                                       MPI_Barrier()            0.010333   
3                                     MPI_Comm_rank()               0.001   
4                                     MPI_Comm_size()                 0.0   
5                                      MPI_Finalize()               0.464   
6                                          MPI_Init()            1.005333   
7                                         MPI_Irecv()            0.012333   
8                                         MPI_Isend()            0.121667   
9                                        MPI_Reduce()               0.008   
10                                         MPI_Wait()            4.357667   
11                                      MPI_Waitall()               8.978   